In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from sqlalchemy.orm import sessionmaker
from modules.config import CITY_NAME
from modules.get_city_graph import get_city_graph

# 1. Define Steel Aréna Parking Coordinates (in WGS84)
parking_coords = (48.715342, 21.246145)  # lat, lon
parking_point = Point(parking_coords[1], parking_coords[0])  # lon, lat

nodes, edges_gdf = get_city_graph(city_name = CITY_NAME)

# 2. Ensure edges_gdf has correct CRS (EPSG:4326)
if edges_gdf.crs is None:
    edges_gdf.set_crs(epsg=4326, inplace=True)

# 3. Reproject both edges and point to EPSG:32634 (UTM zone 34N)
edges_projected = edges_gdf.to_crs(epsg=32634)
parking_projected = gpd.GeoSeries([parking_point], crs='EPSG:4326').to_crs(epsg=32634).iloc[0]

# 4. Compute distance from point to each edge
edges_projected['distance_to_parking'] = edges_projected.geometry.distance(parking_projected)

# 5. Find the closest edge
min_idx = edges_projected['distance_to_parking'].idxmin()
nearest_edge = edges_projected.loc[min_idx]

# 6. Print result
print(f"Nearest edge to Steel Arena parking is edge_id={nearest_edge['edge_id']} at {nearest_edge['distance_to_parking']:.2f} meters.")
